<a href="https://colab.research.google.com/github/WhiteTeaDragon/SingularValues/blob/main/Validate_Theorem_6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import tensorflow as tf

k = 3
n = 16
x_axis = sorted(set(np.around(np.logspace(0, 3, dtype=int))))

In [2]:
# strided_method from https://stackoverflow.com/questions/43735034/create-a-matrix-from-a-vector-where-each-row-is-a-shifted-version-of-the-vector
def strided_method(ar):
    a = np.concatenate((ar[1:], ar))
    L = len(ar)
    n = a.strides[0]
    return np.lib.stride_tricks.as_strided(a[L-1:], (L,L), (-n,n)).copy()

def strided_method_for_matrices(arr):
    a = np.concatenate((arr[1:], arr))
    L = len(arr)
    n = a.strides[0]
    return np.lib.stride_tricks.as_strided(a[L-1:], (L, L, *(arr[0].shape)), (-n,n, *(a.strides[1:])))

def build_doubly_block_circ_matrix(k, n, kernel2d):
    padded_kernel = np.pad(kernel2d, [(0, n - k), (0, n - k)], mode='constant')
    arr = [0] * n
    for i in range(n):
        arr[i] = strided_method(padded_kernel[i])
    return strided_method_for_matrices(arr).transpose(0, 2, 1, 3).reshape(-1, n**2)

def build_matrix(k, n, m, kernel):
    dbc_matrices = [[0] * m for i in range(m)]
    for i in range(m):
        for j in range(m):
            dbc_matrices[i][j] = build_doubly_block_circ_matrix(k, n, kernel[:, :, i, j])
    return np.block(dbc_matrices)

In [7]:
def simple_method(matr, kernel, n):
    return np.linalg.svd(matr, compute_uv=False)

def numpy_method(matr, kernel, n):
    transforms = np.fft.fft2(kernel, (n, n), axes=[0, 1])
    return np.linalg.svd(transforms, compute_uv=False)

def tensorflow_method128(matr, kernel, n):
    conv_tr = tf.cast(tf.transpose(kernel, perm=[2, 3, 0, 1]), tf.complex128)
    transforms = tf.transpose(
        tf.signal.fft2d(
            tf.pad(
                conv_tr, ((0, 0), (0, 0), (0, n - kernel.shape[0]),
                                (0, n - kernel.shape[0]))
            )
        ), 
        perm=[2, 3, 0, 1]
    )
    return tf.linalg.svd(transforms, compute_uv=False)

def tensorflow_method64(matr, kernel, n):
    conv_tr = tf.cast(tf.transpose(kernel, perm=[2, 3, 0, 1]), tf.complex64)
    transforms = tf.transpose(
        tf.signal.fft2d(
            tf.pad(
                conv_tr, ((0, 0), (0, 0), (0, n - kernel.shape[0]),
                                (0, n - kernel.shape[0]))
            )
        ), 
        perm=[2, 3, 0, 1]
    )
    return tf.linalg.svd(transforms, compute_uv=False)

In [8]:
for m in x_axis[:13]:
    print("Starting with", m)
    kernel = np.random.standard_normal(size=(k, k, m, m))
    matr = build_matrix(k, n, m, kernel)
    simple_svd = simple_method(matr, kernel, n)
    numpy_svd = np.array(sorted(list(numpy_method(matr, kernel, n).flatten()), reverse=True))
    tensorflow_svd = np.array(sorted(list(tf.reshape(
        tensorflow_method128(matr, kernel, n), [n * n * m])), reverse=True))
    tensorflow64_svd = np.array(sorted(list(tf.reshape(
        tensorflow_method64(matr, kernel, n), [n * n * m])), reverse=True))
    simple_numpy = np.allclose(simple_svd, numpy_svd)
    numpy_tensorflow = np.allclose(numpy_svd, tensorflow_svd)
    simple_tensorflow = np.allclose(simple_svd, tensorflow_svd)
    simple_tensorflow64 = np.allclose(simple_svd, tensorflow64_svd)
    print("SIMPLE close to NUMPY", simple_numpy)
    print("NUMPY close to TENSORFLOW_128", numpy_tensorflow)
    print("SIMPLE close to TENSORFLOW_128", simple_tensorflow)
    print("SIMPLE close to TENSORFLOW_64", simple_tensorflow64)
    if not simple_numpy or not numpy_tensorflow or not simple_tensorflow:
        print("ERROR!!!")
        break

Starting with 1
SIMPLE close to NUMPY True
NUMPY close to TENSORFLOW_128 True
SIMPLE close to TENSORFLOW_128 True
SIMPLE close to TENSORFLOW_64 True
Starting with 2
SIMPLE close to NUMPY True
NUMPY close to TENSORFLOW_128 True
SIMPLE close to TENSORFLOW_128 True
SIMPLE close to TENSORFLOW_64 True
Starting with 3
SIMPLE close to NUMPY True
NUMPY close to TENSORFLOW_128 True
SIMPLE close to TENSORFLOW_128 True
SIMPLE close to TENSORFLOW_64 True
Starting with 4
SIMPLE close to NUMPY True
NUMPY close to TENSORFLOW_128 True
SIMPLE close to TENSORFLOW_128 True
SIMPLE close to TENSORFLOW_64 True
Starting with 5
SIMPLE close to NUMPY True
NUMPY close to TENSORFLOW_128 True
SIMPLE close to TENSORFLOW_128 True
SIMPLE close to TENSORFLOW_64 True
Starting with 6
SIMPLE close to NUMPY True
NUMPY close to TENSORFLOW_128 True
SIMPLE close to TENSORFLOW_128 True
SIMPLE close to TENSORFLOW_64 True
Starting with 7
SIMPLE close to NUMPY True
NUMPY close to TENSORFLOW_128 True
SIMPLE close to TENSORFLOW_1

In [ ]:
kernel.dtype

dtype('float64')

In [ ]:
(kernel + 0j).dtype

dtype('complex128')